In [121]:
import numpy as np
import pandas as pd
import warnings
import copy
import random
from scipy.special import logsumexp
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedShuffleSplit
warnings.filterwarnings("ignore")

# Read in Dataset

In [49]:
df = pd.read_csv("./codon_usage.csv")

In [50]:
df.head()

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13028 entries, 0 to 13027
Data columns (total 69 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Kingdom      13028 non-null  object 
 1   DNAtype      13028 non-null  int64  
 2   SpeciesID    13028 non-null  int64  
 3   Ncodons      13028 non-null  int64  
 4   SpeciesName  13028 non-null  object 
 5   UUU          13028 non-null  object 
 6   UUC          13028 non-null  object 
 7   UUA          13028 non-null  float64
 8   UUG          13028 non-null  float64
 9   CUU          13028 non-null  float64
 10  CUC          13028 non-null  float64
 11  CUA          13028 non-null  float64
 12  CUG          13028 non-null  float64
 13  AUU          13028 non-null  float64
 14  AUC          13028 non-null  float64
 15  AUA          13028 non-null  float64
 16  AUG          13028 non-null  float64
 17  GUU          13028 non-null  float64
 18  GUC          13028 non-null  float64
 19  GUA 

# Preprocessing

### Drop Faulty Points

In [52]:
# lines 488 and 5065 are index 486 and 5063
df = df.drop([486, 5063], axis = 0)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13027
Data columns (total 69 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Kingdom      13026 non-null  object 
 1   DNAtype      13026 non-null  int64  
 2   SpeciesID    13026 non-null  int64  
 3   Ncodons      13026 non-null  int64  
 4   SpeciesName  13026 non-null  object 
 5   UUU          13026 non-null  object 
 6   UUC          13026 non-null  object 
 7   UUA          13026 non-null  float64
 8   UUG          13026 non-null  float64
 9   CUU          13026 non-null  float64
 10  CUC          13026 non-null  float64
 11  CUA          13026 non-null  float64
 12  CUG          13026 non-null  float64
 13  AUU          13026 non-null  float64
 14  AUC          13026 non-null  float64
 15  AUA          13026 non-null  float64
 16  AUG          13026 non-null  float64
 17  GUU          13026 non-null  float64
 18  GUC          13026 non-null  float64
 19  GUA 

### Drop columns 2-5 (starting at 1)

In [54]:
# drop unused features
vals = [1, 2, 3, 4]
df = df.drop(df.columns[vals], axis = 1)

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13027
Data columns (total 65 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Kingdom  13026 non-null  object 
 1   UUU      13026 non-null  object 
 2   UUC      13026 non-null  object 
 3   UUA      13026 non-null  float64
 4   UUG      13026 non-null  float64
 5   CUU      13026 non-null  float64
 6   CUC      13026 non-null  float64
 7   CUA      13026 non-null  float64
 8   CUG      13026 non-null  float64
 9   AUU      13026 non-null  float64
 10  AUC      13026 non-null  float64
 11  AUA      13026 non-null  float64
 12  AUG      13026 non-null  float64
 13  GUU      13026 non-null  float64
 14  GUC      13026 non-null  float64
 15  GUA      13026 non-null  float64
 16  GUG      13026 non-null  float64
 17  GCU      13026 non-null  float64
 18  GCC      13026 non-null  float64
 19  GCA      13026 non-null  float64
 20  GCG      13026 non-null  float64
 21  CCU      130

In [56]:
df.head()

,Kingdom,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


### Cast two object columns to the correct type

In [57]:
df['UUU'] = df['UUU'].astype(float)

In [58]:
df['UUC'] = df['UUC'].astype(float)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13026 entries, 0 to 13027
Data columns (total 65 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Kingdom  13026 non-null  object 
 1   UUU      13026 non-null  float64
 2   UUC      13026 non-null  float64
 3   UUA      13026 non-null  float64
 4   UUG      13026 non-null  float64
 5   CUU      13026 non-null  float64
 6   CUC      13026 non-null  float64
 7   CUA      13026 non-null  float64
 8   CUG      13026 non-null  float64
 9   AUU      13026 non-null  float64
 10  AUC      13026 non-null  float64
 11  AUA      13026 non-null  float64
 12  AUG      13026 non-null  float64
 13  GUU      13026 non-null  float64
 14  GUC      13026 non-null  float64
 15  GUA      13026 non-null  float64
 16  GUG      13026 non-null  float64
 17  GCU      13026 non-null  float64
 18  GCC      13026 non-null  float64
 19  GCA      13026 non-null  float64
 20  GCG      13026 non-null  float64
 21  CCU      130

In [60]:
len(df)

13026

### Split into target labels and the data points

In [61]:
y = df.iloc[:, 0]

In [62]:
# target labels
y

0        vrl
1        vrl
2        vrl
3        vrl
4        vrl
        ... 
13023    pri
13024    pri
13025    pri
13026    pri
13027    pri
Name: Kingdom, Length: 13026, dtype: object

In [63]:
x = df.iloc[:, 1:]

In [64]:
# 64 dim data points
x

,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,...,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,...,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,...,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,...,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,...,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,...,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,...,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13024,0.01258,0.03193,0.01984,0.00629,0.01451,0.05322,0.07644,0.01258,0.03096,0.06386,...,0.00145,0.00000,0.00048,0.00194,0.01306,0.01838,0.00677,0.00242,0.00097,0.01887
13025,0.01423,0.03321,0.01661,0.00356,0.01127,0.05042,0.09609,0.01068,0.02728,0.06643,...,0.00000,0.00000,0.00000,0.00178,0.01661,0.02788,0.00297,0.00356,0.00119,0.02017
13026,0.01757,0.02028,0.00767,0.01293,0.01319,0.01959,0.00715,0.03964,0.01600,0.02082,...,0.01142,0.01217,0.01196,0.02178,0.02510,0.02896,0.03959,0.00099,0.00079,0.00156


### Convert to numpy object

In [65]:
x = x.to_numpy()

In [66]:
x

array([[0.01654, 0.01203, 0.0005 , ..., 0.00251, 0.0005 , 0.     ],
       [0.02714, 0.01357, 0.00068, ..., 0.00271, 0.00068, 0.     ],
       [0.01974, 0.0218 , 0.01357, ..., 0.00391, 0.     , 0.00144],
       ...,
       [0.01423, 0.03321, 0.01661, ..., 0.00356, 0.00119, 0.02017],
       [0.01757, 0.02028, 0.00767, ..., 0.00099, 0.00079, 0.00156],
       [0.01778, 0.03724, 0.01732, ..., 0.00156, 0.00114, 0.02161]])

### Scale Data points between 0-1

In [67]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [68]:
x.min()

0.0

In [69]:
x.max()

1.0

**turn y into numpy object**

In [165]:
y = y.to_numpy()

In [166]:
y

array(['vrl', 'vrl', 'vrl', ..., 'pri', 'pri', 'pri'], dtype=object)

# Get True Clustering

In [167]:
true_clusters = {}
for i in range(len(y)):
    if y[i] not in true_clusters.keys():
        true_clusters[y[i]] = []
        true_clusters[y[i]].append(x[i])
    else:
        true_clusters[y[i]].append(x[i])

In [169]:
for key in true_clusters.keys():
    print(len(true_clusters[key]))

2831
126
2919
220
18
2523
1345
2077
572
215
180


# Functions to implement DENCLUE algorithm

In [170]:
def NMI():
    pass



In [171]:
'''
x - density attractor
D - dataset
h - user-defined width param
'''
def GetDensity(x, D, h):
    n = len(D)
    d = len(D[0])
    
    coeff = 1 / (n * np.power(h, d))
    
    summation = 0
    for i in range(n):
        z = x - D[i]
        z = z / h
        res = GaussianK(z, d)
        summation += res
    
    
    r = coeff * summation
    return r



In [209]:
'''
Get density attractor for a point
x - point
D - dataset
h - user-defined width param
epsilon - break condition
'''
def FindAttractor(point, D, y, h, epsilon):
    t = 0
    x = copy.deepcopy(point)
    x_old = copy.deepcopy(point)
    
    # sample 20% of the data
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    indices = None
    for train_index, test_index in sss.split(D, y):
        indices = test_index
    # now indices will be an array of shuffles indices 20% of the size of D
    
    while(True):
        # calculate the numerator of x_t+1
        num = np.zeros(len(point))
        for i in range(len(indices)):
            z = x - D[indices[i]]
            z = z / h
            res = GaussianK(z, len(point))
            prod = res * x
            num += prod
        # calculate the denominator of x_t+1
        denom = 0 
        for i in range(len(indices)):
            z = x - D[indices[i]]
            z = z / h
            res = GaussianK(z, len(point))
            denom += res
        
        # calculate x_t+1
        x = num / denom
        
        t += 1
        
        # check break condition
        if np.linalg.norm(x - x_old) <= epsilon:
            break
        else:
            # save x into old x before the next iteration
            x_old = copy.deepcopy(x)
            
        
    return x
        
    




In [173]:
'''
Compute the gaussian kernel under the assumption that the cov matrix 
is the identity matrix
d - num of dimensions
z - d dim vector
'''
def GaussianK(z, d):
    coeff = 1 / np.power(2 * np.pi, d / 2)
    expo = np.exp(-1 * np.dot(z.T, z) / 2)
    
    return coeff * expo
    


In [214]:
'''
D - Dataset
h - user-defined width param
xi - minimum density threshold for a ensity attractor
epsilon - break condition
theta - distance threthold for density reachable attractors
'''
def DENCLUE(D, y, h, xi, epsilon, theta):
    attractors = []
    
    for i in range(len(D) // 20):
        attr = FindAttractor(D[i], D, y, h, epsilon)
        
        # check if the attractor is al least the minimum density
        density = GetDensity(attr, D, h)
        print(density)
        if density < xi:
            continue
        else:
            attractors.append(attr)
        
    print(len(D) // 20)
    print(len(attractors))
#         else:
#             # only add non-duplicate attractor points
#             if len(attractors) == 0:
#                 attractors.append(attr)
#             elif any(np.equal(np.array(attractors),attr).all(1)) == False:
#                 attractors.append(attr)
        
        # print(np.linalg.norm(attr - D[i]))
        

In [ ]:
DENCLUE(x, y, 0.25, 2e11, 1e-2, 1e-3)

13004338935.618866
12870403604.672024
271242405446.6804
214113992089.9127
157260681196.08942
123408828169.69305
166562565670.44254
313411322866.9478
13509878937.060452
218921778617.8604
219303117368.48868
221861657974.50458
207262882839.5746
97537851498.14577
98844943128.48666
84102541372.48933
77283354171.82808
7738862808.56739
16474816378.208836
8449603534.896198
16824426532.719446
53768631170.47946
18905080514.27882
26771472582.28483
15811045706.623861
13356771396.468351
115984356344.98872
193420706377.78235
251407333604.8156
196489130261.7748
56435693077.4072
200995481674.30438
120276929136.34329
391908576168.2087
785871966.4764448
141013183187.90643
194776095300.02994
227167932519.99182
141488195677.07776
190995868332.31976
289125857123.8405
384982745348.13257
107304204700.01442
291996961794.8943
89824874159.114
496667842721.2091
335746327721.3522
274577569508.2434
362828878804.0199
24648391852.38358
175622275267.2841
51605331790.497086
25680716190.794186
74817637515.47414
8030792